# 데이터 분석
> 1. 각 컬럼별 관중 수 관계 파악하기
> 2. 컬럼별 관중 수 평균으로 군집화
> 3. 상관계수 확인하기

In [101]:
# 계산
import pandas as pd
import numpy as np

# 차트
import seaborn as sns
import matplotlib.pyplot as plt

# 표준화
from sklearn.preprocessing import StandardScaler

In [49]:
# 차트 경고 메세지 무시
import warnings
warnings.filterwarnings('ignore')

In [50]:
# 한글 폰트 문제 해결 함수
def fontKorea():
    # 한글 폰트 문제 해결 
    # matplotlib은 한글 폰트를 지원하지 않음
    # os정보
    import platform

    # font_manager : 폰트 관리 모듈
    # rc : 폰트 변경 모듈
    from matplotlib import font_manager, rc
    # unicode 설정
    plt.rcParams['axes.unicode_minus'] = False

    if platform.system() == 'Darwin':
        rc('font', family='AppleGothic') # os가 macos
    elif platform.system() == 'Windows':
        path = 'c:/Windows/Fonts/malgun.ttf' # os가 windows
        font_name = font_manager.FontProperties(fname=path).get_name()
        rc('font', family=font_name)
    else:
        print("Unknown System")

In [51]:
# 데이터 열기
kiwoom_new = pd.read_csv('./Data/kiwoom_new.csv')
kiwoom_new.head()

,날짜,팀명,순위,총 경기수,승리,패배,무승부,승률,게임차,최근 10경기 전적,...,홈팀,원정팀,장소,관중 수,시작시간,종료시간,경기시간,홈팀 선발,어웨이팀 선발,구장
0,2022-04-02,키움,6,1,0,1,0,0.000,1.0,0승0무1패,...,키움,롯데,고척,8257,14:00,17:12,3:12,안우진,반즈,고척
1,2022-04-03,키움,4,2,1,1,0,0.500,1.0,1승0무1패,...,키움,롯데,고척,6115,14:00,17:55,3:55,요키시,박세웅,고척
2,2022-04-05,키움,6,3,1,2,0,0.333,2.0,1승0무2패,...,키움,LG,고척,2298,18:30,21:34,3:04,애플러,임찬규,고척
3,2022-04-06,키움,7,4,1,3,0,0.250,3.0,1승0무3패,...,키움,LG,고척,2304,18:30,22:13,3:43,최원태,손주영,고척
4,2022-04-07,키움,7,5,1,4,0,0.200,4.0,1승0무4패,...,키움,LG,고척,2055,18:30,21:01,2:31,정찬헌,김윤식,고척


In [52]:
kiwoom_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 171 entries, 0 to 170
Data columns (total 24 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   날짜          171 non-null    object 
 1   팀명          171 non-null    object 
 2   순위          171 non-null    int64  
 3   총 경기수       171 non-null    int64  
 4   승리          171 non-null    int64  
 5   패배          171 non-null    int64  
 6   무승부         171 non-null    int64  
 7   승률          171 non-null    float64
 8   게임차         171 non-null    float64
 9   최근 10경기 전적  171 non-null    object 
 10  연속 승패 현황    171 non-null    object 
 11  홈 경기 전적     171 non-null    object 
 12  원정 경기 전적    171 non-null    object 
 13  요일          171 non-null    object 
 14  홈팀          171 non-null    object 
 15  원정팀         171 non-null    object 
 16  장소          171 non-null    object 
 17  관중 수        171 non-null    int64  
 18  시작시간        171 non-null    object 
 19  종료시간        171 non-null    o

----
## 필요 데이터 선별
----

In [53]:
# 날짜 데이터 타입 변환
kiwoom_new['날짜'] = kiwoom_new['날짜'].astype('datetime64[ns]')
kiwoom_new

,날짜,팀명,순위,총 경기수,승리,패배,무승부,승률,게임차,최근 10경기 전적,...,홈팀,원정팀,장소,관중 수,시작시간,종료시간,경기시간,홈팀 선발,어웨이팀 선발,구장
0,2022-04-02,키움,6,1,0,1,0,0.000,1.0,0승0무1패,...,키움,롯데,고척,8257,14:00,17:12,3:12,안우진,반즈,고척
1,2022-04-03,키움,4,2,1,1,0,0.500,1.0,1승0무1패,...,키움,롯데,고척,6115,14:00,17:55,3:55,요키시,박세웅,고척
2,2022-04-05,키움,6,3,1,2,0,0.333,2.0,1승0무2패,...,키움,LG,고척,2298,18:30,21:34,3:04,애플러,임찬규,고척
3,2022-04-06,키움,7,4,1,3,0,0.250,3.0,1승0무3패,...,키움,LG,고척,2304,18:30,22:13,3:43,최원태,손주영,고척
4,2022-04-07,키움,7,5,1,4,0,0.200,4.0,1승0무4패,...,키움,LG,고척,2055,18:30,21:01,2:31,정찬헌,김윤식,고척
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166,2024-05-19,키움,7,45,19,26,0,0.422,10.0,4승0무6패,...,키움,SSG,고척,10049,14:01,17:13,3:12,김인범,오원석,고척
167,2024-05-21,키움,8,46,19,27,0,0.413,10.0,4승0무6패,...,키움,NC,고척,3940,18:31,21:53,3:22,하영민,신민혁,고척
168,2024-05-22,키움,8,47,19,28,0,0.404,10.0,4승0무6패,...,키움,NC,고척,4337,18:31,21:54,3:23,후라도,하트,고척
169,2024-05-23,키움,8,48,20,28,0,0.417,9.0,5승0무5패,...,키움,NC,고척,3897,18:30,21:14,2:44,헤이수스,김시훈,고척


In [54]:
kiwoom_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 171 entries, 0 to 170
Data columns (total 24 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   날짜          171 non-null    datetime64[ns]
 1   팀명          171 non-null    object        
 2   순위          171 non-null    int64         
 3   총 경기수       171 non-null    int64         
 4   승리          171 non-null    int64         
 5   패배          171 non-null    int64         
 6   무승부         171 non-null    int64         
 7   승률          171 non-null    float64       
 8   게임차         171 non-null    float64       
 9   최근 10경기 전적  171 non-null    object        
 10  연속 승패 현황    171 non-null    object        
 11  홈 경기 전적     171 non-null    object        
 12  원정 경기 전적    171 non-null    object        
 13  요일          171 non-null    object        
 14  홈팀          171 non-null    object        
 15  원정팀         171 non-null    object        
 16  장소          171 non-null  

In [55]:
# 해당 열들은 목표로 하는 분석 내용과 상관이 없는 데이터 -> 삭제
kiwoom_new = kiwoom_new.drop(['팀명','홈팀','장소','구장','홈팀 선발','어웨이팀 선발'],axis=1)
kiwoom_new

,날짜,순위,총 경기수,승리,패배,무승부,승률,게임차,최근 10경기 전적,연속 승패 현황,홈 경기 전적,원정 경기 전적,요일,원정팀,관중 수,시작시간,종료시간,경기시간
0,2022-04-02,6,1,0,1,0,0.000,1.0,0승0무1패,1패,0-0-1,0-0-0,토,롯데,8257,14:00,17:12,3:12
1,2022-04-03,4,2,1,1,0,0.500,1.0,1승0무1패,1승,1-0-1,0-0-0,일,롯데,6115,14:00,17:55,3:55
2,2022-04-05,6,3,1,2,0,0.333,2.0,1승0무2패,1패,1-0-2,0-0-0,화,LG,2298,18:30,21:34,3:04
3,2022-04-06,7,4,1,3,0,0.250,3.0,1승0무3패,2패,1-0-3,0-0-0,수,LG,2304,18:30,22:13,3:43
4,2022-04-07,7,5,1,4,0,0.200,4.0,1승0무4패,3패,1-0-4,0-0-0,목,LG,2055,18:30,21:01,2:31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166,2024-05-19,7,45,19,26,0,0.422,10.0,4승0무6패,1승,10-0-14,9-0-12,일,SSG,10049,14:01,17:13,3:12
167,2024-05-21,8,46,19,27,0,0.413,10.0,4승0무6패,1패,10-0-15,9-0-12,화,NC,3940,18:31,21:53,3:22
168,2024-05-22,8,47,19,28,0,0.404,10.0,4승0무6패,2패,10-0-16,9-0-12,수,NC,4337,18:31,21:54,3:23
169,2024-05-23,8,48,20,28,0,0.417,9.0,5승0무5패,1승,11-0-16,9-0-12,목,NC,3897,18:30,21:14,2:44


----
## 2022~2023년 데이터와 2024년 데이터 분리
----

In [56]:
# 2022 ~ 2023년도 데이터
kiwoom_2022_2023 = \
    kiwoom_new[
        (kiwoom_new.날짜.dt.year == 2022) | 
        (kiwoom_new.날짜.dt.year == 2023)
        ]
kiwoom_2022_2023

,날짜,순위,총 경기수,승리,패배,무승부,승률,게임차,최근 10경기 전적,연속 승패 현황,홈 경기 전적,원정 경기 전적,요일,원정팀,관중 수,시작시간,종료시간,경기시간
0,2022-04-02,6,1,0,1,0,0.000,1.0,0승0무1패,1패,0-0-1,0-0-0,토,롯데,8257,14:00,17:12,3:12
1,2022-04-03,4,2,1,1,0,0.500,1.0,1승0무1패,1승,1-0-1,0-0-0,일,롯데,6115,14:00,17:55,3:55
2,2022-04-05,6,3,1,2,0,0.333,2.0,1승0무2패,1패,1-0-2,0-0-0,화,LG,2298,18:30,21:34,3:04
3,2022-04-06,7,4,1,3,0,0.250,3.0,1승0무3패,2패,1-0-3,0-0-0,수,LG,2304,18:30,22:13,3:43
4,2022-04-07,7,5,1,4,0,0.200,4.0,1승0무4패,3패,1-0-4,0-0-0,목,LG,2055,18:30,21:01,2:31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138,2023-09-21,10,134,53,78,3,0.405,27.0,2승0무8패,2승,30-2-35,23-1-43,목,NC,5178,18:30,21:18,2:48
139,2023-09-28,10,136,55,78,3,0.414,27.0,4승0무6패,4승,31-2-35,24-1-43,목,SSG,7082,17:00,19:59,2:59
140,2023-09-29,10,137,55,79,3,0.410,27.0,4승0무6패,1패,31-2-36,24-1-43,금,KIA,14472,14:00,17:57,3:57
141,2023-10-07,10,141,57,81,3,0.413,27.5,6승0무4패,2승,32-2-36,25-1-45,토,LG,13301,17:00,19:49,2:49


In [57]:
# 2024년도 데이터
kiwoom_2024 = \
    kiwoom_new[
        (kiwoom_new.날짜.dt.year == 2024)
        ].reset_index(drop=True)
kiwoom_2024

,날짜,순위,총 경기수,승리,패배,무승부,승률,게임차,최근 10경기 전적,연속 승패 현황,홈 경기 전적,원정 경기 전적,요일,원정팀,관중 수,시작시간,종료시간,경기시간
0,2024-03-29,10,4,0,4,0,0.000,4.0,0승0무4패,4패,0-0-1,0-0-3,금,LG,8465,18:30,21:13,2:43
1,2024-03-30,8,5,1,4,0,0.200,4.0,1승0무4패,1승,1-0-1,0-0-3,토,LG,13462,17:01,19:45,2:44
2,2024-03-31,7,6,2,4,0,0.333,4.0,2승0무4패,2승,2-0-1,0-0-3,일,LG,11822,14:00,16:49,2:49
3,2024-04-05,5,9,5,4,0,0.556,2.5,5승0무4패,5승,3-0-1,2-0-3,금,한화,16000,18:30,21:31,3:01
4,2024-04-06,5,10,6,4,0,0.600,1.5,6승0무4패,6승,4-0-1,2-0-3,토,한화,16000,17:01,19:56,2:55
5,2024-04-07,3,11,7,4,0,0.636,1.0,7승0무3패,7승,5-0-1,2-0-3,일,한화,16000,14:00,17:25,3:25
6,2024-04-12,3,15,9,6,0,0.600,2.5,8승0무2패,2승,6-0-1,3-0-5,금,롯데,8838,18:30,21:31,3:01
7,2024-04-13,3,16,10,6,0,0.625,2.5,8승0무2패,3승,7-0-1,3-0-5,토,롯데,14197,17:00,19:52,2:52
8,2024-04-14,3,17,11,6,0,0.647,2.5,8승0무2패,4승,8-0-1,3-0-5,일,롯데,12333,14:00,17:45,3:45
9,2024-04-16,2,18,12,6,0,0.667,1.5,8승0무2패,5승,9-0-1,3-0-5,화,KT,2877,18:31,21:04,2:33


----
## 컬럼별 타겟 분석 시작
----

----
### 순위별
----

In [59]:
# 순위 데이터만 가져오기
rank_crowd = kiwoom_2022_2023[['순위','관중 수']]
rank_crowd

,순위,관중 수
0,6,8257
1,4,6115
2,6,2298
3,7,2304
4,7,2055
...,...,...
138,10,5178
139,10,7082
140,10,14472
141,10,13301


In [71]:
rank_crowd['순위'].value_counts()

순위
2     27
3     25
8     15
10    15
6     14
9     14
4     13
5      9
7      7
1      4
Name: count, dtype: int64

In [74]:
# 데이터 준비 (예시)
data_series = rank_crowd['순위'].value_counts()

# 데이터 최소값 및 최대값 계산
min_value = data_series.min()
max_value = data_series.max()

# 0~1 사이 표준화
normalized_data = (data_series - min_value) / (max_value - min_value)

# 결과 출력
print(normalized_data)

순위
2     1.000000
3     0.913043
8     0.478261
10    0.478261
6     0.434783
9     0.434783
4     0.391304
5     0.217391
7     0.130435
1     0.000000
Name: count, dtype: float64


In [88]:
# 변환
rank_mapping = {
    normalized_data.index[0] : normalized_data[2],
    normalized_data.index[1] : normalized_data[3],
    normalized_data.index[2] : normalized_data[8],
    normalized_data.index[3] : normalized_data[10],
    normalized_data.index[4] : normalized_data[6],
    normalized_data.index[5] : normalized_data[9],
    normalized_data.index[6] : normalized_data[4],
    normalized_data.index[7] : normalized_data[5],
    normalized_data.index[8] : normalized_data[7],
    normalized_data.index[9] : normalized_data[1]
}
# 변환 적용
kiwoom_2022_2023['순위'].replace(rank_mapping, inplace=True)
kiwoom_2024['순위'].replace(rank_mapping, inplace=True)

----
### 연속 승패 현황
----

In [99]:
# 순위 데이터만 가져오기
consecutive_crowd = kiwoom_2022_2023[['연속 승패 현황','관중 수']]
consecutive_crowd

,연속 승패 현황,관중 수
0,1패,8257
1,1승,6115
2,1패,2298
3,2패,2304
4,3패,2055
...,...,...
138,2승,5178
139,4승,7082
140,1패,14472
141,2승,13301


In [100]:
consecutive_crowd['연속 승패 현황'].value_counts()

연속 승패 현황
1패    33
1승    31
2승    19
2패    10
3승    10
3패     8
4승     7
4패     7
5패     4
6패     4
5승     2
6승     2
7승     2
7패     2
8승     1
9패     1
Name: count, dtype: int64

In [102]:
# 데이터 준비 (예시)
data_series = consecutive_crowd['연속 승패 현황'].value_counts()

# 데이터 최소값 및 최대값 계산
min_value = data_series.min()
max_value = data_series.max()

# 0~1 사이 표준화
normalized_data = (data_series - min_value) / (max_value - min_value)

# 결과 출력
print(normalized_data)

연속 승패 현황
1패    1.00000
1승    0.93750
2승    0.56250
2패    0.28125
3승    0.28125
3패    0.21875
4승    0.18750
4패    0.18750
5패    0.09375
6패    0.09375
5승    0.03125
6승    0.03125
7승    0.03125
7패    0.03125
8승    0.00000
9패    0.00000
Name: count, dtype: float64


In [105]:
normalized_data.index[0]

'1패'

In [106]:
# 변환
consecutive_mapping = {
    normalized_data.index[0] : normalized_data[0],
    normalized_data.index[1] : normalized_data[1],
    normalized_data.index[2] : normalized_data[2],
    normalized_data.index[3] : normalized_data[3],
    normalized_data.index[4] : normalized_data[4],
    normalized_data.index[5] : normalized_data[5],
    normalized_data.index[6] : normalized_data[6],
    normalized_data.index[7] : normalized_data[7],
    normalized_data.index[8] : normalized_data[8],
    normalized_data.index[9] : normalized_data[9],
    normalized_data.index[10] : normalized_data[10],
    normalized_data.index[11] : normalized_data[11],
    normalized_data.index[12] : normalized_data[12],
    normalized_data.index[13] : normalized_data[13],
    normalized_data.index[14] : normalized_data[14],
    normalized_data.index[15] : normalized_data[15]
}
# 변환 적용
kiwoom_2022_2023['연속 승패 현황'].replace(consecutive_mapping, inplace=True)
kiwoom_2024['연속 승패 현황'].replace(consecutive_mapping, inplace=True)

In [107]:
kiwoom_2022_2023[['관중 수','연속 승패 현황','순위']]

,관중 수,연속 승패 현황,순위
0,8257,1.00000,0.434783
1,6115,0.93750,0.391304
2,2298,1.00000,0.434783
3,2304,0.28125,0.130435
4,2055,0.21875,0.130435
...,...,...,...
138,5178,0.56250,0.478261
139,7082,0.18750,0.478261
140,14472,1.00000,0.478261
141,13301,0.56250,0.478261


----
### 타깃 표준화
----

In [110]:
kiwoom_2022_2023['관중 수']

0       8257
1       6115
2       2298
3       2304
4       2055
       ...  
138     5178
139     7082
140    14472
141    13301
142    11757
Name: 관중 수, Length: 143, dtype: int64

In [111]:
# 데이터 준비 (예시)
data_series = kiwoom_2022_2023['관중 수']

# 데이터 최소값 및 최대값 계산
min_value = data_series.min()
max_value = data_series.max()

# 0~1 사이 표준화
normalized_data = (data_series - min_value) / (max_value - min_value)

# 결과 출력
print(normalized_data)

0      0.491462
1      0.350782
2      0.100092
3      0.100486
4      0.084132
         ...   
138    0.289242
139    0.414291
140    0.899645
141    0.822737
142    0.721332
Name: 관중 수, Length: 143, dtype: float64


In [112]:
kiwoom_2022_2023['관중 수'] = normalized_data
kiwoom_2022_2023

,날짜,순위,총 경기수,승리,패배,무승부,승률,게임차,최근 10경기 전적,연속 승패 현황,홈 경기 전적,원정 경기 전적,요일,원정팀,관중 수,시작시간,종료시간,경기시간
0,2022-04-02,0.434783,1,0,1,0,0.000,1.0,0승0무1패,1.00000,0-0-1,0-0-0,토,롯데,0.491462,14:00,17:12,3:12
1,2022-04-03,0.391304,2,1,1,0,0.500,1.0,1승0무1패,0.93750,1-0-1,0-0-0,일,롯데,0.350782,14:00,17:55,3:55
2,2022-04-05,0.434783,3,1,2,0,0.333,2.0,1승0무2패,1.00000,1-0-2,0-0-0,화,LG,0.100092,18:30,21:34,3:04
3,2022-04-06,0.130435,4,1,3,0,0.250,3.0,1승0무3패,0.28125,1-0-3,0-0-0,수,LG,0.100486,18:30,22:13,3:43
4,2022-04-07,0.130435,5,1,4,0,0.200,4.0,1승0무4패,0.21875,1-0-4,0-0-0,목,LG,0.084132,18:30,21:01,2:31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138,2023-09-21,0.478261,134,53,78,3,0.405,27.0,2승0무8패,0.56250,30-2-35,23-1-43,목,NC,0.289242,18:30,21:18,2:48
139,2023-09-28,0.478261,136,55,78,3,0.414,27.0,4승0무6패,0.18750,31-2-35,24-1-43,목,SSG,0.414291,17:00,19:59,2:59
140,2023-09-29,0.478261,137,55,79,3,0.410,27.0,4승0무6패,1.00000,31-2-36,24-1-43,금,KIA,0.899645,14:00,17:57,3:57
141,2023-10-07,0.478261,141,57,81,3,0.413,27.5,6승0무4패,0.56250,32-2-36,25-1-45,토,LG,0.822737,17:00,19:49,2:49


In [114]:
kiwoom_2022_2023[['관중 수','순위','연속 승패 현황']].corr()

,관중 수,순위,연속 승패 현황
관중 수,1.000000,-0.220005,0.023085
순위,-0.220005,1.000000,0.039926
연속 승패 현황,0.023085,0.039926,1.000000


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

In [ ]:
# k-mean로 관중 수 범위 나눌 k 값 찾기

# 데이터 전처리: 타깃 컬럼과 피쳐 컬럼 분리
features = kiwoom_2022_2023[['요일','순위']]
target = kiwoom_2022_2023[['관중 수']]

attendance_shape = np.array(target).reshape(-1,1)

# 특성 표준화
scaler = StandardScaler()
features_scaled = scaler.fit_transform(attendance_shape)

# 엘보우 방법을 사용하여 최적의 클러스터 수 찾기
sse = []
k_range = range(3, 15)  # 3부터 15까지의 k 값에 대해 확인

for k in k_range:
    kmeans = KMeans(n_clusters=k)
    kmeans.fit(attendance_shape)
    sse.append(kmeans.inertia_)

# 엘보우 그래프 그리기
plt.figure(figsize=(10, 6))
plt.plot(k_range, sse, marker='o')
plt.xlabel('Number of clusters (k)')
plt.ylabel('Sum of squared distances (SSE)')
plt.title('Elbow Method For Optimal k')
plt.grid(True)
plt.show()


In [ ]:
# K-means 클러스터링
kmeans = KMeans(n_clusters=5)
clusters = kmeans.fit_predict(attendance_shape)
clusters

# 각 클러스터의 관중 수 범위 계산
cluster_ranges = {}

for cluster in range(5):
    cluster_data = \
        kiwoom_2022_2023[clusters == cluster]
    min_attendance = \
        cluster_data['관중 수'].min()
    max_attendance = \
        cluster_data['관중 수'].max()
    cluster_ranges[cluster] = (min_attendance, max_attendance)
cluster_ranges

In [ ]:
# 규칙에 따라 관중 수를 분류하여 새로운 컬럼에 할당
kiwoom_2022_2023['관중수 분류'] = \
    pd.cut(kiwoom_2022_2023['관중 수'], 
    bins=[0, 3300, 6000, 9100, 12900, float('inf')], 
    labels=[
        np.mean(cluster_ranges[1]).round(0),
        np.mean(cluster_ranges[4]).round(0),
        np.mean(cluster_ranges[2]).round(0),
        np.mean(cluster_ranges[0]).round(0),
        np.mean(cluster_ranges[3]).round(0)
        ], 
    right=False)

In [ ]:
# # 규칙에 따라 관중 수를 분류하여 새로운 컬럼에 할당
# kiwoom_2022_2023['관중수 분류'] = \
#     pd.cut(kiwoom_2022_2023['관중 수'], 
#     bins=[0, 3300, 6000, 9100, 12900, float('inf')], 
#     labels=[
#         2,
#         5,
#         3,
#         1,
#         4
#         ], 
#     right=False)

In [ ]:
kiwoom_2022_2023['관중수 분류'] = kiwoom_2022_2023['관중수 분류'].astype(int)
kiwoom_2022_2023

In [ ]:
# 규칙에 따라 관중 수를 분류하여 새로운 컬럼에 할당
kiwoom_2024['관중수 분류'] = \
    pd.cut(kiwoom_2024['관중 수'], 
    bins=[0, 3300, 6000, 9100, 12900, float('inf')], 
    labels=[
        np.mean(cluster_ranges[1]).round(0),
        np.mean(cluster_ranges[4]).round(0),
        np.mean(cluster_ranges[2]).round(0),
        np.mean(cluster_ranges[0]).round(0),
        np.mean(cluster_ranges[3]).round(0)
        ], 
    right=False)

In [ ]:
# # 규칙에 따라 관중 수를 분류하여 새로운 컬럼에 할당
# kiwoom_2024['관중수 분류'] = \
#     pd.cut(kiwoom_2024['관중 수'], 
#     bins=[0, 3300, 6000, 9100, 12900, float('inf')], 
#     labels=[
#         2,
#         5,
#         3,
#         1,
#         4
#         ], 
#     right=False)

In [ ]:
kiwoom_2024['관중수 분류'] = kiwoom_2024['관중수 분류'].astype(int)
kiwoom_2024

In [ ]:
kiwoom_2022_2023[['순위','요일','관중 수']]

In [ ]:
kiwoom_2022_2023[['관중 수','관중수 분류','순위','요일']].corr()

In [ ]:
kiwoom_2022_2023.info()

In [ ]:
kiwoom_2024[['순위','요일','관중수 분류']]

In [ ]:
kiwoom_2024.info()

In [116]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

In [115]:
# 데이터 준비 (x는 피쳐, y는 타겟)
x = kiwoom_2022_2023[['연속 승패 현황','순위']]
y = kiwoom_2022_2023['관중 수']

# 실제 데이터 준비
test_x = kiwoom_2024[['연속 승패 현황','순위']]
test_y = kiwoom_2024['관중 수']

In [117]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

# 랜덤 포레스트 회귀 모델 학습
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# 예측하기
predicted_prices = model.predict(X_test)

# 평균 제곱 오차 계산
mse = mean_squared_error(y_test, predicted_prices)
print('MSE:', mse)

# 루트 평균 제곱 오차 계산
rmse = mean_squared_error(y_test, predicted_prices)**0.5
print('RMSE:', rmse)

# 평균 절대 오차 계산
mae = mean_absolute_error(y_test, predicted_prices)
print('MAE:', mae)

# 전체 데이터 범위 계산
data_range = max(y) - min(y)

# 백분율 계산 (MAE 기준)
accuracy_percentage_mae = (1 - (mae / data_range)) * 100
print('정확도 (백분율, MAE 기준):', accuracy_percentage_mae)

# 백분율 계산 (RMSE 기준)
accuracy_percentage_rmse = (1 - (rmse / data_range)) * 100
print('정확도 (백분율, RMSE 기준):', accuracy_percentage_rmse)

MSE: 0.06471595785553236
RMSE: 0.2543933133074302
MAE: 0.202849852838301
정확도 (백분율, MAE 기준): 79.71501471616989
정확도 (백분율, RMSE 기준): 74.56066866925698


In [118]:
# 랜덤 포레스트 회귀 모델 학습
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(x, y)

# 예측하기
predicted_prices = model.predict(test_x)

# 평균 제곱 오차 계산
mse = mean_squared_error(test_y, predicted_prices)
print('MSE:', mse)

# 루트 평균 제곱 오차 계산
rmse = mean_squared_error(test_y, predicted_prices)**0.5
print('RMSE:', rmse)

# 평균 절대 오차 계산
mae = mean_absolute_error(test_y, predicted_prices)
print('MAE:', mae)

# 전체 데이터 범위 계산
data_range = max(test_y) - min(test_y)

# 백분율 계산 (MAE 기준)
accuracy_percentage_mae = (1 - (mae / data_range)) * 100
print('정확도 (백분율, MAE 기준):', accuracy_percentage_mae)

# 백분율 계산 (RMSE 기준)
accuracy_percentage_rmse = (1 - (rmse / data_range)) * 100
print('정확도 (백분율, RMSE 기준):', accuracy_percentage_rmse)

MSE: 111231996.29926507
RMSE: 10546.658063067422
MAE: 9674.5317742703
정확도 (백분율, MAE 기준): 26.278047898572744
정확도 (백분율, RMSE 기준): 19.63226348344569


In [120]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR

In [122]:
# 랜덤 포레스트 회귀 모델 학습
model = DecisionTreeRegressor()
model.fit(x, y)

# 예측하기
predicted_prices = model.predict(test_x)

# 평균 제곱 오차 계산
mse = mean_squared_error(test_y, predicted_prices)
print('MSE:', mse)

# 루트 평균 제곱 오차 계산
rmse = mean_squared_error(test_y, predicted_prices)**0.5
print('RMSE:', rmse)

# 평균 절대 오차 계산
mae = mean_absolute_error(test_y, predicted_prices)
print('MAE:', mae)

# 전체 데이터 범위 계산
data_range = max(test_y) - min(test_y)

# 백분율 계산 (MAE 기준)
accuracy_percentage_mae = (1 - (mae / data_range)) * 100
print('정확도 (백분율, MAE 기준):', accuracy_percentage_mae)

# 백분율 계산 (RMSE 기준)
accuracy_percentage_rmse = (1 - (rmse / data_range)) * 100
print('정확도 (백분율, RMSE 기준):', accuracy_percentage_rmse)

MSE: 111232470.37694772
RMSE: 10546.680538299608
MAE: 9674.549053953317
정확도 (백분율, MAE 기준): 26.277916223780263
정확도 (백분율, RMSE 기준): 19.632092217483745


In [123]:
# 랜덤 포레스트 회귀 모델 학습
model = KNeighborsRegressor(n_neighbors=5)
model.fit(x, y)

# 예측하기
predicted_prices = model.predict(test_x)

# 평균 제곱 오차 계산
mse = mean_squared_error(test_y, predicted_prices)
print('MSE:', mse)

# 루트 평균 제곱 오차 계산
rmse = mean_squared_error(test_y, predicted_prices)**0.5
print('RMSE:', rmse)

# 평균 절대 오차 계산
mae = mean_absolute_error(test_y, predicted_prices)
print('MAE:', mae)

# 전체 데이터 범위 계산
data_range = max(test_y) - min(test_y)

# 백분율 계산 (MAE 기준)
accuracy_percentage_mae = (1 - (mae / data_range)) * 100
print('정확도 (백분율, MAE 기준):', accuracy_percentage_mae)

# 백분율 계산 (RMSE 기준)
accuracy_percentage_rmse = (1 - (rmse / data_range)) * 100
print('정확도 (백분율, RMSE 기준):', accuracy_percentage_rmse)

MSE: 111231973.557986
RMSE: 10546.65698494011
MAE: 9674.545999324464
정확도 (백분율, MAE 기준): 26.277939500689907
정확도 (백분율, RMSE 기준): 19.632271699000913


In [124]:
# 랜덤 포레스트 회귀 모델 학습
model = SVR(kernel='linear')
model.fit(x, y)

# 예측하기
predicted_prices = model.predict(test_x)

# 평균 제곱 오차 계산
mse = mean_squared_error(test_y, predicted_prices)
print('MSE:', mse)

# 루트 평균 제곱 오차 계산
rmse = mean_squared_error(test_y, predicted_prices)**0.5
print('RMSE:', rmse)

# 평균 절대 오차 계산
mae = mean_absolute_error(test_y, predicted_prices)
print('MAE:', mae)

# 전체 데이터 범위 계산
data_range = max(test_y) - min(test_y)

# 백분율 계산 (MAE 기준)
accuracy_percentage_mae = (1 - (mae / data_range)) * 100
print('정확도 (백분율, MAE 기준):', accuracy_percentage_mae)

# 백분율 계산 (RMSE 기준)
accuracy_percentage_rmse = (1 - (rmse / data_range)) * 100
print('정확도 (백분율, RMSE 기준):', accuracy_percentage_rmse)

MSE: 111231071.7439334
RMSE: 10546.614231303494
MAE: 9674.531107755214
정확도 (백분율, MAE 기준): 26.278052977556854
정확도 (백분율, RMSE 기준): 19.632597490638616


In [121]:
# 의사결정 나무
dtree = DecisionTreeRegressor()
dtree = dtree.fit(X_train, y_train)
print(f"dTree : {dtree.score(X_test, y_test)*100}")

# KNN
knn = KNeighborsRegressor(n_neighbors=5).fit(X_train, y_train)
print(f"kNN : {knn.score(X_test, y_test)*100}")

# SVM
svm = SVR(kernel='linear').fit(X_train, y_train)
print(f"svm : {svm.score(X_test, y_test)*100}")

dTree : -79.92859315392104
kNN : 6.993566046797984
svm : 1.2838857577706086


In [ ]:
from sklearn.ensemble import VotingClassifier
voting_clf = VotingClassifier(
    estimators=[
        ('decision_tree', dtree),
        ('knn', knn),
        ('svm', svm),
    ],
    weights=[1,1,1],
    voting='hard'
).fit(X_train, Y_train)

voting_clf.score(X_test, Y_test)*100

In [ ]:
voting_clf = VotingClassifier(
    estimators=[
        ('decision_tree', dtree),
        ('knn', knn),
        ('svm', svm),
    ],
    weights=[1,1,1],
    voting='soft'
).fit(X_train, Y_train)

voting_clf.score(X_test, Y_test)*100